<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo5/1_ab_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1hiUFVbQ2Jgrv0olU-pcf89ODJ5t2pRVe" width="100%">

# Evaluación de Modelos
---

En este notebook veremos un ejemplo de experimentación y evaluación de modelos ya desplegados por medio de técnicas estadísticas. Comenzamos importando las librerías necesarias:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from scipy import stats
plt.style.use("ggplot")

La evaluación de modelos es un proceso importante para garantizar que los modelos de aprendizaje automático estén funcionando correctamente en un entorno de producción y para detectar cualquier problema o desviación en el rendimiento del modelo.

Algunas de las técnicas comunes para evaluar modelos después del despliegue incluyen:

- **Monitoreo de la precisión**: se puede realizar un seguimiento de la precisión del modelo en tiempo real y establecer alertas para notificar al equipo si la precisión del modelo cae por debajo de un umbral aceptable.
- **Monitoreo de la calidad de los datos**: se puede realizar un seguimiento de la calidad de los datos que ingresan al modelo para detectar cualquier problema o cambio en los patrones de los datos que puedan afectar el rendimiento del modelo.
- **Monitoreo de los resultados del modelo**: se pueden realizar pruebas y experimentos continuos para evaluar los resultados del modelo y compararlos con los resultados esperados.
- **Evaluación de la calidad de los resultados**: se pueden realizar pruebas y experimentos para evaluar la calidad de los resultados del modelo y compararlos con los resultados esperados.
- **Verificación de la equidad del modelo**: se pueden realizar pruebas para verificar si el modelo está produciendo resultados equitativos para diferentes grupos de usuarios o datos.

En general, la evaluación continua de los modelos de aprendizaje automático después del despliegue es esencial para garantizar que los modelos estén funcionando correctamente en producción y para detectar cualquier problema o desviación en el rendimiento del modelo. En este notebook veremos el detalle de A/B testing enfocado en la evaluación de modelos.

## **1. A/B Testing**
---

A/B testing, también conocido como prueba A/B, es una técnica de experimentación que se utiliza para comparar dos versiones diferentes de una página web, aplicación móvil, correo electrónico e incluso un modelo de machine learning. Se utiliza para evaluar cuál de las dos versiones proporciona mejores resultados en términos de una métrica predefinida o key performance indicator (KPI), como la tasa de clics, la tasa de conversión, el tiempo de permanencia, ingresos totales, entre otros.

<img src="https://drive.google.com/uc?export=view&id=1AVLtYO4j--exx3RsWJd33_Wheo9y7WRu" width="80%">

En un A/B test, se divide aleatoriamente a los usuarios en dos grupos: el grupo A y el grupo B. A cada grupo se le muestra una versión diferente del elemento que se está evaluando y se mide el rendimiento de cada versión en función de la métrica predefinida.

<img src="https://drive.google.com/uc?export=view&id=1Xji338rFFvDvmZi7G-7YMeVJa3Gs8Qtf" width="80%">

Luego, se compara el rendimiento de las dos versiones para determinar cuál es mejor. Si una versión proporciona un mejor rendimiento que la otra, se puede seleccionar como la versión óptima para usar en producción. Respecto a aplicaciones de machine learning, A/B testing se utiliza para comparar el rendimiento de dos o más modelos de aprendizaje automático en un entorno de producción en tiempo real. Se utiliza para evaluar qué modelo proporciona mejores resultados para una tarea específica.

## **2. Comparación de Modelos**
---

Veamos un ejemplo con un conjunto de datos obtenidos del siguiente experimento:

<img src="https://drive.google.com/uc?export=view&id=1cy-aV8Vy6xn6RL0KmX9Vv7lvCQlwFFiZ" width="80%">

- Hacemos parte de una compañía fabricante de galletas e implementamos una metodología de generación de precios diferenciales en dependencia del perfil de los distintos puntos de venta.
- Originalmente, implementamos un modelo basado en redes neuronales (**modelo A**) que demostró buenos resultados.
- Junto al equipo de ciencia de datos implementamos una nueva versión del modelo (**modelo B**) que presentó un mejor ajuste durante el entrenamiento y validación cruzada.
- Para validar si el nuevo modelo realmente funciona mejor, se realizó un A/B testing sobre una muestra de 5000 clientes durante un mes, buscamos validar qué modelo genera un mayor ingreso neto (en dólares).

Vamos a cargar el conjunto de datos:

In [ ]:
data = pd.read_parquet("https://raw.githubusercontent.com/mindlab-unal/mlds6-datasets/main/u5/ab_testing.parquet")
display(data.head())

En este caso disponemos de tres columnas:

- `client`: identificador único de cada cliente de la muestra.
- `group`: específica qué modelo fue utilizado con dicho usuario.
- `revenue`: ingresos netos obtenidos de un cliente durante el mes del experimento.

## **3. Evaluación Estadística**
---

Normalmente, los experimentos de tipo A/B testing consisten en una comparación de dos muestras estadísticas, comenzaremos validando qué distribución siguen los datos. Para esto generaremos una gráfica de tipo kernel density estimation para visualizar la forma de ambas distribuciones, para esto, utilizaremos la librería `seaborn`:

In [ ]:
import seaborn as sns

Ahora, generamos la gráfica:

In [ ]:
fig, ax = plt.subplots()
sns.histplot(data, x="revenue", hue="group", ax=ax, kde=True)
fig.show()

Como podemos ver, pareciera que los datos siguiesen una distribución normal. Esto lo podemos corroborar con una prueba de bondad por medio de la prueba de Kolmogorov-Smirnov, para ello, definimos dos distribuciones para cada grupo:

In [ ]:
revenue_a = data.query("group == 'A'").revenue
revenue_b = data.query("group == 'B'").revenue
dist_a = stats.norm(loc=revenue_a.mean(), scale=revenue_a.std())
dist_b = stats.norm(loc=revenue_b.mean(), scale=revenue_b.std())

Veamos el resultado de la prueba de bondad:

In [ ]:
print(stats.kstest(revenue_a, dist_a.cdf))
print(stats.kstest(revenue_b, dist_b.cdf))

Recordemos la hipótesis manejada por la prueba de Kolmogorov-Smirnov:

- $H_0$: los datos siguen la distribución normal.
- $H_1$: los datos no siguen la distribución normal.

Si establecemos un nivel de significancia del `0.05` y como obtenemos `pvalue` que es considerablemente mayor que el nivel de significancia podemos concluir que:

> No podemos rechazar la hipótesis nula $H_0$, es decir, es probable que los datos sigan la distribución normal.

Teniendo en cuenta esto, procedemos a realizar una prueba de comparación de medias para validar qué modelo genera un mayor ingreso neto. Para esto, utilizaremos una prueba de t-student para validar la siguiente hipótesis:

- $H_0$: $\mu_a <= \mu_b$ (en promedio el modelo **A** genera un menor ingreso neto que el modelo **B**).
- $H_1$: $\mu_a > \mu_b$ (en promedio el modelo **A** genera un mayor ingreso neto que el modelo **B**).

Aplicamos la prueba:

In [ ]:
print(stats.ttest_ind(revenue_a, revenue_b, alternative="less"))

Si tomamos un nivel de significancia del `0.05` y al comparar el resultado obtenido de `pvalue`, llegamos a la siguiente conclusión:

> Rechazamos la hipótesis nula, es decir, el modelo **B** probablemente está generando mayores ingresos en comparación con el modelo **A**.

Por último, podemos validar cuánto fue el incremento promedio en los ingresos netos del modelo **B**:

In [ ]:
print(revenue_b.mean() - revenue_a.mean())

Esto nos muestra que el modelo genera en promedio `200` dólares más por cliente, por ende, lo recomendable sería comenzar a utilizar este modelo y sustituir la versión del modelo anterior.

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [A/B Testing](https://es.wikipedia.org/wiki/Prueba_A/B)
- [Kolmogorov-Smirnov test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html)
- [T-student test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**
  - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandra-superlano-02b74313a/).
  - [Mario Andrés Rodríguez Triana](mailto:mrodrigueztr@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*